In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import time
import numpy as np

In [3]:
import pydrake
from pydrake.all import BsplineTrajectoryThroughUnionOfHPolyhedra, IrisInConfigurationSpace, IrisOptions
from pydrake.common import FindResourceOrThrow
from pydrake.geometry import SceneGraph
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.optimization import CalcGridPointsOptions, Toppra
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.multibody.tree import RevoluteJoint
from pydrake.solvers.mathematicalprogram import MathematicalProgram, Solve
from pydrake.solvers.mosek import MosekSolver
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.primitives import TrajectorySource
from pydrake.trajectories import PiecewisePolynomial
from pydrake.all import Variable, Expression, RotationMatrix
from pydrake.all import MultibodyPositionToGeometryPose, ConnectMeshcatVisualizer, Role, Sphere
from pydrake.all import (
    ConvexSet, HPolyhedron, Hyperellipsoid,
    MathematicalProgram, Solve, le, IpoptSolver,
    Role, Sphere, VPolytope,
    Iris, IrisOptions, MakeIrisObstacles, Variable
)
from pydrake.all import (
    eq, SnoptSolver,
    Sphere, Ellipsoid, GeometrySet,
    RigidBody_, AutoDiffXd, initializeAutoDiff,
)

import pydrake.symbolic as sym
import symbolic_parsing_helpers as symHelpers

from meshcat import Visualizer

# Setup meshcat
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

# Sporadically need to run `pkill -f meshcat`

from symbolic_parsing_helpers import generate_rationalize_trig_expr_rules
from symbolic_parsing_helpers import xreplace
from symbolic_parsing_helpers import NotRationalFunctionException
# from symbolic_parsing_helpers import rationalize_trig_expr
from pydrake.all import RationalFunction
snopt = SnoptSolver()
def xreplace(expr, rules):
        if isinstance(expr, float) or isinstance(expr, sym.Variable):
            return expr
        assert isinstance(expr, sym.Expression), expr
        for old, new in rules:
            if expr.EqualTo(old):
                return new
        ctor, old_args = expr.Deconstruct()
        new_args = [xreplace(e, rules) for e in old_args]
        return ctor(*new_args)

# Build simulation Environment

In [4]:
simple_collision = True
# gripper_welded = True

vis = Visualizer(zmq_url=zmq_url)
vis.delete()
display(vis.jupyter_cell())

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
            "drake/manipulation/models/wsg_50_description/package.xml")))

directives_file = FindResourceOrThrow("drake/sandbox/planar_iiwa_simple_collision_welded_gripper.yaml") \
    if simple_collision else FindResourceOrThrow("drake/sandbox/planar_iiwa_dense_collision_welded_gripper.yaml")
directives = LoadModelDirectives(directives_file)
models = ProcessModelDirectives(directives, plant, parser)

q0 = [-0.2, -1.2, 1.6]
index = 0
for joint_index in plant.GetJointIndices(models[0].model_instance):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1

plant.Finalize()

visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, delete_prefix_on_load=False)

diagram = builder.Build()
visualizer.load()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
diagram.Publish(context)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.


In [5]:
def visualize_trajectory(traj):
    builder = DiagramBuilder()

    scene_graph = builder.AddSystem(SceneGraph())
    plant = MultibodyPlant(time_step=0.0)
    plant.RegisterAsSourceForSceneGraph(scene_graph)
    parser = Parser(plant)
    parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
                "drake/manipulation/models/wsg_50_description/package.xml")))

    directives_file = FindResourceOrThrow("drake/sandbox/planar_iiwa_simple_collision_welded_gripper.yaml") \
        if simple_collision else FindResourceOrThrow("drake/sandbox/planar_iiwa_dense_collision_welded_gripper.yaml")
    directives = LoadModelDirectives(directives_file)
    models = ProcessModelDirectives(directives, plant, parser)

    q0 = [-0.2, -1.2, 1.6]
    index = 0
    for joint_index in plant.GetJointIndices(models[0].model_instance):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[index])
            index += 1

    plant.Finalize()

    to_pose = builder.AddSystem(MultibodyPositionToGeometryPose(plant))
    builder.Connect(to_pose.get_output_port(), scene_graph.get_source_pose_port(plant.get_source_id()))

    traj_system = builder.AddSystem(TrajectorySource(traj))
    builder.Connect(traj_system.get_output_port(), to_pose.get_input_port())

    meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url)

    vis_diagram = builder.Build()

    simulator = Simulator(vis_diagram)
    meshcat.start_recording()
    simulator.AdvanceTo(traj.end_time())
    meshcat.publish_recording()
    with open("/tmp/spp_shelves.html", "w") as f:
        f.write(meshcat.vis.static_html())


# Construct Rational Kinematics

In [67]:
from pydrake.all import RationalForwardKinematics
forward_kin = RationalForwardKinematics(plant)
q_star = np.zeros(forward_kin.t().shape[0])
t = forward_kin.t()
def convert_RationalForwardPoseList_to_TransformExpressionList(pose_list):
    ret = []
    for p in pose_list:
        ret.append(p.asRigidTransformExpr())
    return ret


# Define Iris In T-Space

In [7]:
#  Now IRIS in configuration space, using dReal to solve for the growth volume
# through the nonconvex kinematics.

from pydrake.all import (
    eq, SnoptSolver,
    Sphere, Ellipsoid, GeometrySet,
    RigidBody_, AutoDiffXd, initializeAutoDiff,
)


def TangentPlane(self, point):
    a = 2 * self.A().T @ self.A() @ (point - self.center())
    a = a / np.linalg.norm(a)
    b = a.dot(point)
    return a, b

Hyperellipsoid.TangentPlane = TangentPlane


query = scene_graph.get_query_output_port().Eval(scene_graph.GetMyContextFromRoot(context))

sym_plant = plant.ToSymbolic()
sym_context = sym_plant.CreateDefaultContext()

# For SNOPT test.
autodiff_plant = plant.ToAutoDiffXd()
autodiff_context = autodiff_plant.CreateDefaultContext()
snopt = SnoptSolver()

def MakeFromHPolyhedronOrEllipseSceneGraph(query, geom, expressed_in=None):
    shape = query.inspector().GetShape(geom)
    if isinstance(shape, (Sphere, Ellipsoid)):
        return Hyperellipsoid(query, geom, expressed_in)
    return HPolyhedron(query, geom, expressed_in)
def MakeFromVPolytopeOrEllipseSceneGraph(query, geom, expressed_in=None):
    shape = query.inspector().GetShape(geom)
    if isinstance(shape, (Sphere, Ellipsoid)):
        return Hyperellipsoid(query, geom, expressed_in)
    return VPolytope(query, geom, expressed_in)


dReal_polytope_tol = .1

def CheckVolume(E, bodyA, bodyB, setA, setB, A, b, volume):
    prog = MathematicalProgram()
    q = prog.NewContinuousVariables(plant.num_positions(), "q")
    prog.AddBoundingBoxConstraint(lb, ub, q)
    prog.AddLinearConstraint(A, b-np.inf, b, q)
    p_AA =  prog.NewContinuousVariables(3, "p_AA")
    p_BB = prog.NewContinuousVariables(3, "p_BB")
    if volume < np.inf:
        prog.AddConstraint((q-E.center()).T @ E.A().T @ E.A() @ (q-E.center()) <= volume)
    setA.AddPointInSetConstraints(prog, p_AA)
    setB.AddPointInSetConstraints(prog, p_BB)

    sym_plant.SetPositions(sym_context, q)
    X_WA = sym_plant.EvalBodyPoseInWorld(sym_context, bodyA)
    X_WB = sym_plant.EvalBodyPoseInWorld(sym_context, bodyB)
    # Add +0 pending https://github.com/RobotLocomotion/drake/issues/15216
    print(type(X_WA))
    p_WA = X_WA.multiply(p_AA+0)
    p_WB = X_WB.multiply(p_BB+0)
    prog.AddConstraint(eq(p_WA, p_WB))
    prog.SetSolverOption(dReal.id(), "precision", .9*dReal_polytope_tol)
    result = dReal.Solve(prog)
    return result.is_success(), result.GetSolution(q)


def GrowthVolume(E, bodyA, bodyB, setA, setB, A, b, guess=None):
    prog = MathematicalProgram()
    q = prog.NewContinuousVariables(plant.num_positions(), "q")

    if guess is not None:
        prog.SetInitialGuess(q, guess)

    prog.AddLinearConstraint(A, b-np.inf, b, q)
    p_AA = prog.NewContinuousVariables(3, "p_AA")
    p_BB = prog.NewContinuousVariables(3, "p_BB")
    setA.AddPointInSetConstraints(prog, p_AA)
    setB.AddPointInSetConstraints(prog, p_BB)
    prog.AddQuadraticErrorCost(E.A().T @ E.A(), E.center(), q)

    # TODO: Remove these.  They're for debugging only.
    #set_meshcat_object(vis['setA'],setA)
    #set_meshcat_object(vis['setB'],setB)
    print(type(p_AA[0]))
    if isinstance(bodyA, RigidBody_[AutoDiffXd]):
        # TODO: Update this to use MBP<double> and Jacobians if I'm going to keep it.
        def kinematics_constraint(vars):
            p_AA, p_BB, q = np.split(vars,[3,6])
            autodiff_plant.SetPositions(autodiff_context, q)
            X_WA = autodiff_plant.EvalBodyPoseInWorld(autodiff_context, bodyA)
            X_WB = autodiff_plant.EvalBodyPoseInWorld(autodiff_context, bodyB)
            p_WA = X_WA.multiply(p_AA)
            p_WB = X_WB.multiply(p_BB)
            return p_WA - p_WB
        prog.AddConstraint(kinematics_constraint, lb=[0,0,0], ub=[0,0,0],
                           vars=np.concatenate((p_AA,p_BB,q)))
        result = snopt.Solve(prog)

    else:
        # TODO: Construct these symbolic expressions once per body outside this method.
        # But I would have to substitute in the new q each time.
        # Better is to construct the prog once for each pair, and just update the cost?
        sym_plant.SetPositions(sym_context, q)
        X_WA = sym_plant.EvalBodyPoseInWorld(sym_context, bodyA)
        X_WB = sym_plant.EvalBodyPoseInWorld(sym_context, bodyB)
        # Add +0 pending https://github.com/RobotLocomotion/drake/issues/15216
        p_WA = X_WA.multiply(p_AA+0)
        
        p_WB = X_WB.multiply(p_BB+0)
        prog.AddConstraint(eq(p_WA, p_WB))
        result = snopt.Solve(prog)
        #result = dReal.Solve(prog)

    return result.is_success(), result.get_optimal_cost(), result.GetSolution(q)

def iris_cspace(query, point, require_containment_points=[], termination_threshold=2e-2, iteration_limit=100):
#     vis = Visualizer(zmq_url=zmq_url)
#     set_meshcat_object(vis['sample'], point, color=0x99dd99)

    ellipsoid_epsilon = 1e-1
    dim = plant.num_positions()
    lb = plant.GetPositionLowerLimits()
    ub = plant.GetPositionUpperLimits()
#     assert dim == 3 # need to update the volume once this changes
    volume_of_unit_sphere = 4.0*np.pi/3.0
    E = Hyperellipsoid(np.eye(3)/ellipsoid_epsilon, point)
#     set_meshcat_object(vis['ellipse'], E)
    best_volume = ellipsoid_epsilon**dim * volume_of_unit_sphere

    inspector = query.inspector()
    pairs = inspector.GetCollisionCandidates()

    P = HPolyhedron.MakeBox(lb, ub)
    A = np.vstack((P.A(), np.zeros((10*len(pairs),3))))  # allow up to 10 faces per pair.
    b = np.concatenate((P.b(), np.zeros(10*len(pairs))))

    geom_ids = inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)
    sets = {geom:MakeFromHPolyhedronOrEllipseSceneGraph(query, geom, inspector.GetFrameId(geom)) for geom in geom_ids}

    use_autodiff = False
    use_dReal = False
    ad_bodies = {geom:autodiff_plant.GetBodyFromFrameId(inspector.GetFrameId(geom)) for geom in geom_ids}
    sym_bodies = {geom:sym_plant.GetBodyFromFrameId(inspector.GetFrameId(geom)) for geom in geom_ids}
    if use_autodiff:
        bodies = ad_bodies
    else:
        bodies = sym_bodies
    print
    iteration = 0
    num_faces = 2*len(lb)
    while True:
        ## Find separating hyperplanes

        for geomA, geomB in pairs:
            print(f"geomA={inspector.GetName(geomA)}, geomB={inspector.GetName(geomB)}")
            # Run snopt at the beginning
            while True:
                success, growth, qstar = GrowthVolume(E,
                    bodies[geomA], bodies[geomB],
                    sets[geomA], sets[geomB], A[:num_faces,:], b[:num_faces] - dReal_polytope_tol, point)
                if success:
                    print(f"snopt_example={qstar}, growth = {growth}")
                    # Add a face to the polytope
                    A[num_faces,:], b[num_faces] = E.TangentPlane(qstar)
                    num_faces += 1
                else:
                    break

            if use_dReal:
                tries = 0
                while True:
                    reachable, counter_example = CheckVolume(
                        E, sym_bodies[geomA], sym_bodies[geomB], sets[geomA], sets[geomB],
                        A[:num_faces,:], b[:num_faces] - dReal_polytope_tol, np.inf)
                    if not reachable:
                        print("unreachable")
                        break
                    else:
                        z = E.A() @ (counter_example - E.center())
                        dreal_growth = z.dot(z)
                        print(f"counter_example = {counter_example}, growth = {dreal_growth}")
                        success, growth, qstar = GrowthVolume(E,
                            bodies[geomA], bodies[geomB],
                            sets[geomA], sets[geomB], A[:num_faces,:], b[:num_faces] - dReal_polytope_tol, counter_example)
                        if success:
                            print(f"snopt_example={qstar}, growth = {growth}")
                            # Add a face to the polytope
                            A[num_faces,:], b[num_faces] = E.TangentPlane(qstar)
                            num_faces += 1
                        if np.all(A[:num_faces,:] @ counter_example <= b[:num_faces] - dReal_polytope_tol):
                            # Then also add the counter-example
                            A[num_faces,:], b[num_faces] = E.TangentPlane(counter_example)
                            num_faces += 1
                    tries += 1

        if any([np.any(A[:num_faces,:] @ q > b[:num_faces]) for q in require_containment_points]):
            print("terminating because a required containment point would have not been contained")
            break

        P = HPolyhedron(A[:num_faces,:],b[:num_faces])
#         set_meshcat_object(vis[f'polytope'], P, wireframe=True)

        E = P.MaximumVolumeInscribedEllipsoid()
#         set_meshcat_object(vis[f'ellipse'], E)
        print(iteration)

        iteration += 1
        if iteration >= iteration_limit:
            break

        volume = volume_of_unit_sphere / np.linalg.det(E.A())
        if volume - best_volume <= termination_threshold:
            break
        best_volume = volume

    return P


In [8]:
def GrowthVolumeRational(E, X_WA, X_WB, setA, setB, A, b, guess=None):
    snopt = SnoptSolver()
    prog = MathematicalProgram()
    t = forward_kin.t()
    prog.AddDecisionVariables(t)
    
    if guess is not None:
        prog.SetInitialGuess(t, guess)

    prog.AddLinearConstraint(A, b-np.inf, b, t)
    p_AA = prog.NewContinuousVariables(3, "p_AA")
    p_BB = prog.NewContinuousVariables(3, "p_BB")
    setA.AddPointInSetConstraints(prog, p_AA)
    setB.AddPointInSetConstraints(prog, p_BB)
    prog.AddQuadraticErrorCost(E.A().T @ E.A(), E.center(), t)

    p_WA = X_WA.multiply(p_AA+0)

    p_WB = X_WB.multiply(p_BB+0)

    prog.AddConstraint(eq(p_WA, p_WB))
    result = snopt.Solve(prog)
    return result.is_success(), result.get_optimal_cost(), result.GetSolution(t)


In [9]:
def iris_rational_space(query, point, require_containment_points=[], termination_threshold=2e-2, iteration_limit=100):
    dReal_polytope_tol = .1
    ellipsoid_epsilon = 1e-1
    dim = plant.num_positions()
    lb = plant.GetPositionLowerLimits()
    rational_lb = forward_kin.ComputeTValue(lb, q_star)
    ub = plant.GetPositionUpperLimits()
    rational_ub = forward_kin.ComputeTValue(ub, q_star)
    volume_of_unit_sphere = 4.0*np.pi/3.0
    E = Hyperellipsoid(np.eye(3)/ellipsoid_epsilon, point)
    best_volume = ellipsoid_epsilon**dim * volume_of_unit_sphere
    
    link_poses_by_body_index_rat_pose = forward_kin.CalcLinkPoses(q_star, 
                                                         plant.world_body().index())
    X_WA_list = convert_RationalForwardPoseList_to_TransformExpressionList(link_poses_by_body_index_rat_pose)
    X_WB_list = convert_RationalForwardPoseList_to_TransformExpressionList(link_poses_by_body_index_rat_pose)

    inspector = query.inspector()
    pairs = inspector.GetCollisionCandidates()

    P = HPolyhedron.MakeBox(rational_lb, rational_ub)
    A = np.vstack((P.A(), np.zeros((10*len(pairs),3))))  # allow up to 10 faces per pair.
    b = np.concatenate((P.b(), np.zeros(10*len(pairs))))

    geom_ids = inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)
    sets = {geom:MakeFromHPolyhedronOrEllipseSceneGraph(query, geom, inspector.GetFrameId(geom)) for geom in geom_ids}
    body_indexes_by_geom_id = {geom:
                               plant.GetBodyFromFrameId(inspector.GetFrameId(geom)).index() for geom in geom_ids} 
    
    #Turn onto true to certify regions using SOS at each iteration.
    certify = False
    # refine polytopes if not certified collision free
    refine = False and certify
        
    iteration = 0
    num_faces = 2*len(lb)
    while True:
        ## Find separating hyperplanes

        for geomA, geomB in pairs:
            print(f"geomA={inspector.GetName(geomA)}, geomB={inspector.GetName(geomB)}")
            # Run snopt at the beginning
            while True:
                X_WA = X_WA_list[int(body_indexes_by_geom_id[geomA])]
                X_WB = X_WB_list[int(body_indexes_by_geom_id[geomB])]
                success, growth, qstar = GrowthVolumeRational(E,
                    X_WA, X_WB,
                    sets[geomA], sets[geomB], A[:num_faces,:], b[:num_faces] - dReal_polytope_tol, point)
                if success:
                    print(f"snopt_example={qstar}, growth = {growth}")
                    # Add a face to the polytope
                    A[num_faces,:], b[num_faces] = E.TangentPlane(qstar)
                    num_faces += 1
                else:
                    break

            if certify:
                pass
            

        if any([np.any(A[:num_faces,:] @ t > b[:num_faces]) for t in require_containment_points]):
            print("terminating because a required containment point would have not been contained")
            break

        P = HPolyhedron(A[:num_faces,:],b[:num_faces])

        E = P.MaximumVolumeInscribedEllipsoid()
        print(iteration)

        iteration += 1
        if iteration >= iteration_limit:
            break

        volume = volume_of_unit_sphere / np.linalg.det(E.A())
        if volume - best_volume <= termination_threshold:
            break
        best_volume = volume

    return P

In [10]:
seed_points = np.array([[0.0, -2.016, 1.975], # in tight
                        [-1, -2, 0.5],        # neutral pose
                        [0.3, -0.8, 0.5],     # above shelf
                        [0.25, -1.6, -0.25],  # in shelf 1
                        [0.07, -1.8, -0.2],   # leaving shelf 1
                        [-0.1, -2, -0.3]])    # out of shelf 1

# traj = PiecewisePolynomial.FirstOrderHold(np.array([0, 1]), np.array([seed_points[4], seed_points[1]]).T)
# visualize_trajectory(traj)

In [44]:

query = scene_graph.get_query_output_port().Eval(scene_graph.GetMyContextFromRoot(context))
regions = []
for i in range(seed_points.shape[0]):
    
    start_time = time.time()
    hpoly = iris_rational_space(query, seed_points[i,:], require_containment_points=[seed_points[i,:]], iteration_limit=100)
#     hpoly = iris_cspace(query, seed_points[i,:], require_containment_points=[seed_points[i,:]], iteration_limit=100)
    ellipse = hpoly.MaximumVolumeInscribedEllipsoid()
    print("Time: %6.2f \tVolume: %6.2f \tCenter:" % (time.time() - start_time, ellipse.Volume()),
          ellipse.center(), flush=True)
    regions.append(hpoly)
print("SUCCESS")

geomA=iiwa::link7, geomB=shelves::top
snopt_example=[ 0.26672972 -0.41852535  1.63205081], growth = 274.0684135179604
geomA=iiwa::link7, geomB=wsg::right_collision
geomA=wsg::left_collision, geomB=wsg::right_collision
geomA=wsg::right_collision, geomB=shelves::shelf_upper
snopt_example=[ 0.13051903 -0.70054686  0.69516705], growth = 338.5424550728913
geomA=wsg::body_collision, geomB=shelves::top
geomA=wsg::body_collision, geomB=shelves::shelf_upper
geomA=wsg::left_collision, geomB=shelves::top
geomA=wsg::right_collision, geomB=shelves::top
geomA=iiwa::link7, geomB=shelves::shelf_lower
snopt_example=[ 0.52015706 -0.77647848  1.63205081], growth = 192.45911044623415
geomA=wsg::body_collision, geomB=shelves::shelf_lower
geomA=wsg::left_collision, geomB=shelves::shelf_lower
snopt_example=[ 0.16163771 -1.3747254   0.1491634 ], growth = 377.1039149121791
geomA=wsg::right_collision, geomB=shelves::shelf_lower
geomA=iiwa::link7, geomB=wsg::left_collision
geomA=iiwa::link7, geomB=shelves::shelf

In [45]:
print(regions)

[<pydrake.geometry.optimization.HPolyhedron object at 0x7fdf1dd94970>, <pydrake.geometry.optimization.HPolyhedron object at 0x7fdf1de827b0>, <pydrake.geometry.optimization.HPolyhedron object at 0x7fdf1dcdabb0>, <pydrake.geometry.optimization.HPolyhedron object at 0x7fdf1db5ed30>, <pydrake.geometry.optimization.HPolyhedron object at 0x7fdf1ddfda30>, <pydrake.geometry.optimization.HPolyhedron object at 0x7fdf1ddeb9b0>]


In [47]:
poly_to_cert = regions[3]
Atmp = poly_to_cert.A()
btmp = poly_to_cert.b()
print(Atmp)


[[ 1.          0.          0.        ]
 [ 0.          1.          0.        ]
 [ 0.          0.          1.        ]
 [-1.         -0.         -0.        ]
 [-0.         -1.         -0.        ]
 [-0.         -0.         -1.        ]
 [-0.11440078  0.99219152  0.04968357]
 [-0.91545136  0.24261775 -0.32106922]
 [-0.08846971  0.77578708  0.62476197]
 [ 0.48664382  0.87225258  0.04851009]
 [ 0.9452202   0.32623649  0.01133659]
 [-0.52280521 -0.37200906  0.76699672]]


# Get all the bodies and rational poses

In [14]:
link_poses_by_body_index_rat_pose = forward_kin.CalcLinkPoses(q_star, 
                                                         plant.world_body().index())
X_WA_list = convert_RationalForwardPoseList_to_TransformExpressionList(link_poses_by_body_index_rat_pose)
X_WB_list = convert_RationalForwardPoseList_to_TransformExpressionList(link_poses_by_body_index_rat_pose)

inspector = query.inspector()
pairs = inspector.GetCollisionCandidates()


geom_ids = inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)
HPolyhedronSets = {geom:MakeFromHPolyhedronOrEllipseSceneGraph(query, geom, inspector.GetFrameId(geom)) for geom in geom_ids}
VPolyhedronSets = {geom:MakeFromVPolytopeOrEllipseSceneGraph(query, geom, inspector.GetFrameId(geom)) for geom in geom_ids}



body_indexes_by_geom_id = {geom:
                           plant.GetBodyFromFrameId(inspector.GetFrameId(geom)).index() for geom in geom_ids} 

geomA, geomB = next(iter(pairs))
X_WA = X_WA_list[int(body_indexes_by_geom_id[geomA])]
X_WB = X_WB_list[int(body_indexes_by_geom_id[geomB])]
for i, p in enumerate(pairs):
    geomA, geomB = p
    X_WA = X_WA_list[int(body_indexes_by_geom_id[geomA])]
    X_WB = X_WB_list[int(body_indexes_by_geom_id[geomB])]
    if i >= 15:
        break
print(X_WA)
print(X_WB)
bodyA, bodyB = VPolyhedronSets[geomA], VPolyhedronSets[geomB]

RigidTransform_[Expression](
  R=RotationMatrix_[Expression]([
    [<Expression "((-6.5358489673231094e-07 - 1.5314282201569325e-15 * t[0] + 7.4058647983745344e-22 * t[1] - 7.4058647983745344e-22 * t[2] - 2.6143395864393851e-06 * (t[0] * t[1]) - 2.9623459193498134e-21 * (t[0] * t[1] * t[2]) + 2.6143395854596679e-06 * (t[0] * t[1] * pow(t[2], 2)) - 1.5314267389839727e-15 * (t[0] * pow(t[1], 2)) - 2.6143395859495265e-06 * (t[0] * pow(t[1], 2) * t[2]) - 1.5314282201569325e-15 * (t[0] * pow(t[1], 2) * pow(t[2], 2)) + 2.6143395859495265e-06 * (t[0] * t[2]) - 1.5314267389839727e-15 * (t[0] * pow(t[2], 2)) - 7.4058647983745326e-22 * (pow(t[0], 2) * t[1]) + 2.6143395859495265e-06 * (pow(t[0], 2) * t[1] * t[2]) + 7.4058647983745326e-22 * (pow(t[0], 2) * t[1] * pow(t[2], 2)) - 6.5358489673231094e-07 * (pow(t[0], 2) * pow(t[1], 2)) - 7.4058647983745326e-22 * (pow(t[0], 2) * pow(t[1], 2) * t[2]) + 6.5358489624245231e-07 * (pow(t[0], 2) * pow(t[1], 2) * pow(t[2], 2)) + 7.4058647983745326e-22 * (pow

In [15]:
vertex1 = X_WA@RigidTransform(RotationMatrix(), bodyA.vertices()[:,0]).translation()
vertex2 = X_WB@RigidTransform(RotationMatrix(),bodyB.vertices()[:,0]).translation()
base_point = X_WB@RigidTransform(RotationMatrix(), bodyB.vertices().mean(axis = 1)).translation()
print(vertex1)
print(vertex2)
print(base_point)
# print(bodyA.PointInSet(basePoint))

[<Expression "( - 0.01 * ((-0.99999999999978639 - 1.1331067786114829e-15 * t[0] + 1.1331077795293539e-15 * t[1] - 1.1331077795293539e-15 * t[2] - 3.9999999999991456 * (t[0] * t[1]) - 4.532431118117415e-15 * (t[0] * t[1] * t[2]) + 3.9999999999991456 * (t[0] * t[1] * pow(t[2], 2)) + 1.1331087804472246e-15 * (t[0] * pow(t[1], 2)) - 3.9999999999991456 * (t[0] * pow(t[1], 2) * t[2]) - 1.1331067786114833e-15 * (t[0] * pow(t[1], 2) * pow(t[2], 2)) + 3.9999999999991456 * (t[0] * t[2]) + 1.1331087804472242e-15 * (t[0] * pow(t[2], 2)) - 1.1331077795293535e-15 * (pow(t[0], 2) * t[1]) + 3.9999999999991456 * (pow(t[0], 2) * t[1] * t[2]) + 1.1331077795293535e-15 * (pow(t[0], 2) * t[1] * pow(t[2], 2)) - 0.99999999999978639 * (pow(t[0], 2) * pow(t[1], 2)) - 1.1331077795293535e-15 * (pow(t[0], 2) * pow(t[1], 2) * t[2]) + 0.99999999999978639 * (pow(t[0], 2) * pow(t[1], 2) * pow(t[2], 2)) + 1.1331077795293535e-15 * (pow(t[0], 2) * t[2]) - 0.99999999999978639 * (pow(t[0], 2) * pow(t[2], 2)) - 3.9999999999

In [38]:
def convert_Expression_Vect_to_Rational(vect):
    ret = vect.copy()
    for i, v in enumerate(vect):
        ret[i] = symHelpers.RationalFunctionFromExpression(v)
    return np.array(ret)


rat_vertex1 = convert_Expression_Vect_to_Rational(vertex1)
rat_vertex2 = convert_Expression_Vect_to_Rational(vertex2)
rat_base_point = convert_Expression_Vect_to_Rational(base_point)
# print(rat_vert[0].denominator())

#randomly check if expressions are equal
tmp1 = rat_vertex1[0].ToExpression()
tmp2 = vertex1[0]
equal = True
for i in range(1000):
    test = 100*np.random.randn(len(tmp2.GetVariables()))
    ret1 = tmp1.Evaluate(dict(zip(tmp1.GetVariables(),test)))
    ret2 = tmp2.Evaluate(dict(zip(tmp2.GetVariables(),test)))
    equal = equal and np.isclose(ret1,ret2)
    if not equal:
        print(ret1)
        print(ret2)
        break
print(equal)
print(tmp1.Expand().EqualTo(tmp2.Expand()))

True
False


In [41]:
def ExprVectIsAllConstant(vect):
    for v in vect:
        if v.get_kind() != sym.ExpressionKind.Constant:
            return False
    return True
def RatVectIsAllConstant(vect):
    for v in vect:
        tmp = v.ToExpression()
        if tmp.get_kind() != sym.ExpressionKind.Constant:
            return False
    return True
print(ExprVectIsAllConstant(vertex1))
print(RatVectIsAllConstant(rat_vertex1))
print(ExprVectIsAllConstant(vertex2))
print(RatVectIsAllConstant(rat_vertex2))

False
False
True
True


In [18]:
def RationalVectHasCommonDenominatorAcrossVect(vect):
    ret = True
    for i in range(vect.shape[0]-1):
        tmp1 = vect[i].denominator()
        tmp2 = vect[i+1].denominator()
        ret = ret and tmp1.CoefficientsAlmostEqual(tmp2, 1e-10)
        ret = tmp1.EqualTo(tmp2)
    return ret

In [19]:
print(tmp2.GetVariables())

{t[0], t[1], t[2]}


In [20]:
prog = MathematicalProgram()
print(prog)
def addSeparatingVertexSeparatingHyperplaneConstraint(prog, vertex1, vertex2, base_point, plane_number = None):
    plane_name = "a" + str(plane_number) if plane_number is not None else "a"
    a = prog.NewContinuousVariables(vertex1.shape[0], plane_name)
    vect1 = vertex1 - base_point
    vect2 = vertex2 - base_point
    prog.AddLinearConstraint(a@vect1 <= 1)
    prog.AddLinearConstraint(a@vect2 >= 1)
    return prog
# prog = addSeparatingVertexSeparatingHyperplaneConstraint(prog, vertex1, vertex2, base_point, plane_number = 1)

a2 = prog.NewContinuousVariables(vertex1.shape[0], "a2")
convSolver = MosekSolver()
result = convSolver.Solve(prog)
print(result.is_success())
print(result.GetSolution())

No decision variables.

True
[0. 0. 0.]


In [89]:
prog = MathematicalProgram()
a2 = prog.NewContinuousVariables(vertex1.shape[0], "a2")
convSolver = MosekSolver()

def polynomialHyperlaneConstraintPolynomial(t, leq_or_geq, rat_vertex, rat_base_point, plane):
    vect = rat_vertex - rat_base_point
    if not RationalVectHasCommonDenominatorAcrossVect(vect):
        raise ValueError("vertex needs to have same denominator across entries")
    numerator_vect = np.array([v.numerator() for v in vect])
    prod = 0
    for i in range(numerator_vect.shape[0]):
        prod += numerator_vect[i]*plane[i]
    if leq_or_geq == 'leq':
        poly_sos_expr = -prod+vect[0].denominator()
    elif leq_or_geq == 'geq':
        poly_sos_expr = prod-vect[0].denominator()
    else:
        raise ValueError('leq_or_geq must be leq or geq not {}'.format(leq_or_geq))
    poly_sos_expr.SetIndeterminates(sym.Variables(t))
    return poly_sos_expr
def polynomialHyperlaneConstraint(prog, t, leq_or_geq, rat_vertex, rat_base_point, plane,
                                  t_Polyhedron_Constraint, lagrange_mult_degree = 4):
    poly_sos_expr = polynomialHyperlaneConstraintPolynomial(t, leq_or_geq, rat_vertex, rat_base_point, plane)
    cone_rhs = 0
    A = t_Polyhedron_Constraint.A()
    b = t_Polyhedron_Constraint.b()
    for i in range(A.shape[0]):
        gi = sym.Polynomial(A[i,:]@t-b[i])
        lagrange_mult_poly, Gram_matrix = prog.NewSosPolynomial(sym.Variables(t), lagrange_mult_degree)
        cone_rhs += lagrange_mult_poly*gi
    prog.AddConstraint(poly_sos_expr == lagrange_mult_poly)
    return prog
def constantHyperplaneConstraint(prog, leq_or_geq, vertex, base_point, plane):
    vect = vertex - base_point
    if leq_or_geq == 'leq':
        prog.AddLinearConstraint(plane@vect <= 1)
    elif leq_or_geq == 'geq':
        prog.AddLinearConstraint(plane@vect >= 1)
    else:
        raise ValueError('leq_or_geq must be leq or geq not {}'.format(leq_or_geq))
    return prog
    
    
poly = polynomialHyperlaneConstraintPolynomial(t, 'leq', rat_vertex1, rat_base_point, a2)
# prog = polynomialHyperlaneConstraint(prog, t, 'leq', rat_vertex1, rat_base_point, a2, poly_to_cert)

prog = constantHyperplaneConstraint(prog, 'geq', vertex2, base_point, a2)
result = convSolver.Solve(prog)
print(result.is_success())
print(result.GetSolution())

True
[-6.66666667  0.          0.        ]


In [37]:
print(poly.decision_variables())
print(poly.indeterminates())


{}
{t[0], t[1], t[2]}
